https://youtu.be/0ZqeBEa_MWo?si=euGFINmBNnY8qPbn&t=149

In [ ]:
import holoviews as hv
import numpy as np
import pandas as pd
from icecream import ic

In [ ]:
hv.extension("bokeh")

In [ ]:
spool_radius = 5.75
gear_ratio = (59.0 / 17.0) ** 2
STEP_DIVISION = 1
motor_steps_per_revolution = 100 * STEP_DIVISION
spool_circumfrence = spool_radius * 2 * np.pi
ic(spool_circumfrence)
steps_per_mm = motor_steps_per_revolution * gear_ratio / spool_circumfrence
ic(steps_per_mm)
max_rpm = 100.0
max_revs_per_second = max_rpm / 60.0
max_steps_per_second = max_revs_per_second * motor_steps_per_revolution
ic(max_steps_per_second)
max_velocity = max_steps_per_second / steps_per_mm
ic(max_velocity)
max_acceleration = 1.0
max_jerk = 1.0

# Stage 0: Constant Jerk

Reach maximum acceleration

acceleration = max_jerk * t

velocity = 0.5 * max_jerk * t ** 2

t_max_jerk_2 = sqrt(velocity * 2 / max_jerk)

position = 1 /6 * max_jerk * t ** 3

In [ ]:
sc = [{"t":0.0, "v":0.0, "p":0.0, "dt": 0.0, "dv": 0.0, "dp": 0.0} for i in range(7)]
# either max_jerk to max_acceleration or max_velocity
sc[0]["dt"] = sc[0]["t"] = min(ic(max_acceleration / max_jerk), ic(np.sqrt(max_velocity * 2 / max_jerk)))
sc[0]["dv"] = sc[0]["v"] = max_jerk * sc[0]["t"] ** 2 / 2
sc[0]["dp"] = sc[0]["p"] = max_jerk * sc[0]["t"] ** 3 / 6
ic(sc[0])

# Stage 1: Constant Acceleration

Boost speed to almost maximum at max acceleration

delta_acc = max_velocity - v_max_jerk * 2

v_max_acc = v_max_jerk + max_acceleration * t_max_acc

p_max_acc = v_max_jerk * t_max_acc + 1 /2 * max_acceleration * t_max_acc ** 2

In [ ]:
sc[1]["dv"] = ic(ic(max_velocity) - ic(v_max_jerk * 2))
sc[1]["v"] = sc[0]["v"] + sc[1]["dv"]
sc[1]["dt"] = sc[1]["dv"] / max_acceleration
sc[1]["t"] = sc[0]["t"] + sc[1]["dt"]
sc[1]["dp"] = ic(sc[0]["v"] * sc[1]["dt"]) + ic(1 /2 * max_acceleration * sc[1]["dt"] ** 2)
sc[1]["p"] = sc[0]["p"] + sc[1]["dp"]
ic(sc[1])

# Stage 2: Constant jerk

Now we need to reduce our acceleration back down to 0 with out final velocity being max_velocity

In [ ]:
# All the constant jerk sections should have the same dt
sc[2]["dt"] = sc[0]["dt"]
sc[2]["dv"] = max_acceleration * sc[2]["dt"] ** 2 / 2
sc[2]["v"] = sc[1]["v"] + sc[2]["dv"]
ic(sc[2])
assert(sc[2]["v"] == max_velocity)

In [ ]:
start = np.array([150.0, 200.0]).reshape(1,2)
end = np.array([0.0, 0.0]).reshape(1,2)